In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("6_features.csv")

In [3]:
X = df.loc[:, df.columns!= 'target']
y = df['target']

In [4]:
from sklearn.linear_model import LogisticRegression
L_R = LogisticRegression(multi_class="multinomial", max_iter=1000)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [6]:
L_R.fit(X_train, y_train)
y_hat_L_R = L_R.predict(X_test)
print(classification_report(y_test, y_hat_L_R))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.96      0.96      0.96     11860
           1       0.94      0.88      0.91      3797
           2       0.94      1.00      0.97      4343

    accuracy                           0.95     20000
   macro avg       0.95      0.94      0.94     20000
weighted avg       0.95      0.95      0.95     20000



In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ('scaler', StandardScaler()),           # ensure all features live on the same scale
    ('clf', LogisticRegression(
        max_iter=500,                       # give solvers more room
        random_state=42                     # reproducibility
    ))
])

param_grid = [
    # L2 only, with two solver choices
    {
      'clf__penalty': ['l2'],
      'clf__C':       [0.1, 1, 10],
      'clf__solver':  ['lbfgs'],             # saga rarely outperforms lbfgs on pure L2
      'clf__class_weight': [None, 'balanced']
    },
    # L1 only (sparse), but only with saga
    {
      'clf__penalty': ['l1'],
      'clf__C':       [1, 10],               # skip 0.1 if it never converges
      'clf__solver':  ['saga'],
      'clf__class_weight': [None, 'balanced']
    }
]

grid = GridSearchCV(
    pipe, param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid.fit(X, y)
print("Best params:", grid.best_params_)
print("CV accuracy:", grid.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max

Best params: {'clf__C': 10, 'clf__class_weight': None, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}
CV accuracy: 0.9587600000000001


In [21]:
LR_GRID = LogisticRegression(C= 10, class_weight= None, penalty= 'l2', solver= 'lbfgs')

In [23]:
LR_GRID.fit(X_train, y_train)
y_hat_LR_GRID = LR_GRID.predict(X_test)
print(classification_report(y_test, y_hat_LR_GRID))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95     11860
           1       0.91      0.89      0.90      3797
           2       0.92      0.99      0.95      4343

    accuracy                           0.94     20000
   macro avg       0.93      0.94      0.93     20000
weighted avg       0.94      0.94      0.94     20000



/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
